<a href="https://colab.research.google.com/github/pawan123-k/my-project/blob/main/FP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install nibabel opencv-python-headless kagglehub

import os
import numpy as np
import nibabel as nib
import cv2
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers, models
import kagglehub

In [ ]:
from google.colab import files
files.upload()  # Select your kaggle.json file here

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"zorojoro","key":"43f23108f5222230862699c91c361bc2"}'}

In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

!kaggle datasets download -d andrewmvd/liver-tumor-segmentation

import zipfile

with zipfile.ZipFile("liver-tumor-segmentation.zip", 'r') as zip_ref:
    zip_ref.extractall("liver_tumor_dataset")

Dataset URL: https://www.kaggle.com/datasets/andrewmvd/liver-tumor-segmentation
License(s): Attribution-NonCommercial-NoDerivatives 4.0 International (CC BY-NC-ND 4.0)
 99% 4.81G/4.84G [00:47<00:00, 151MB/s]
100% 4.84G/4.84G [00:47<00:00, 108MB/s]


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import zipfile

with zipfile.ZipFile("liver-tumor-segmentation.zip", 'r') as zip_ref:
    zip_ref.extractall("liver_tumor_dataset")

from google.colab import drive
drive.mount('/content/drive')

import shutil

# Merge all .nii volumes
merged_volume_path = '/content/liver_tumor_dataset/all_volumes'
os.makedirs(merged_volume_path, exist_ok=True)

for pt in range(1, 6):
    part_path = f'/content/liver_tumor_dataset/volume_pt{pt}'
    if os.path.exists(part_path):
        for file in os.listdir(part_path):
            if file.endswith('.nii'):
                shutil.copy(os.path.join(part_path, file), merged_volume_path)

image_dir = merged_volume_path
label_dir = '/content/liver_tumor_dataset/segmentations'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Load NIfTI images
def load_nii_images(volume_path, seg_path, image_size=(128, 128)):
    if not os.path.exists(volume_path) or not os.path.exists(seg_path):
        return np.empty((0, *image_size, 1)), np.empty((0,))

    vol_files = sorted([f for f in os.listdir(volume_path) if f.endswith('.nii') or f.endswith('.nii.gz')])
    seg_files = sorted([f for f in os.listdir(seg_path) if f.endswith('.nii') or f.endswith('.nii.gz')])

    images, labels = [], []
    for vol_file, seg_file in zip(vol_files, seg_files):
        vol = nib.load(os.path.join(volume_path, vol_file)).get_fdata()
        seg = nib.load(os.path.join(seg_path, seg_file)).get_fdata()

        for slice_idx in range(min(vol.shape[2], seg.shape[2])):
            img_slice = vol[:, :, slice_idx]
            seg_slice = seg[:, :, slice_idx]
            label = 1 if np.any(seg_slice > 0) else 0
            img_resized = cv2.resize(img_slice.astype(np.float32), image_size)
            images.append(img_resized)
            labels.append(label)

    return np.expand_dims(np.array(images), axis=-1), np.array(labels)

X, y = load_nii_images(image_dir, label_dir)

if X.shape[0] == 0 or y.shape[0] == 0:
    X = np.random.rand(100, 128, 128, 1)
    y = np.random.randint(0, 2, 100)

if X.shape[0] > 0 and y.shape[0] > 0:
    X = X / 255.0
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# CNN Model
def build_cnn(input_shape):
    model = models.Sequential()
    model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=input_shape))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Conv2D(64, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Conv2D(128, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Flatten())
    model.add(layers.Dense(128, activation='relu'))
    model.add(layers.Dropout(0.5))
    model.add(layers.Dense(1, activation='sigmoid'))
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

cnn_model = build_cnn(input_shape=X_train.shape[1:])
cnn_model.fit(X_train, y_train, epochs=2, batch_size=32, validation_data=(X_test, y_test))
cnn_accuracy = cnn_model.evaluate(X_test, y_test, verbose=0)[1]
print("CNN Accuracy:", cnn_accuracy)

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/2
430/430 ━━━━━━━━━━━━━━━━━━━━ 461s 1s/step - accuracy: 0.8004 - loss: 0.4624 - val_accuracy: 0.9616 - val_loss: 0.1027
Epoch 2/2
430/430 ━━━━━━━━━━━━━━━━━━━━ 492s 1s/step - accuracy: 0.9543 - loss: 0.1158 - val_accuracy: 0.9918 - val_loss: 0.0295
CNN Accuracy: 0.9918485879898071


In [ ]:
# RNN Model
X_rnn = X.reshape((X.shape[0], X.shape[1], X.shape[2]))
X_train_rnn, X_test_rnn, y_train_rnn, y_test_rnn = train_test_split(X_rnn, y, test_size=0.2, random_state=42)

def build_rnn(input_shape):
    model = models.Sequential([
        layers.Input(shape=input_shape),
        layers.LSTM(64),
        layers.Dense(1, activation='sigmoid')
    ])
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

rnn_model = build_rnn(X_train_rnn.shape[1:])
rnn_model.fit(X_train_rnn, y_train_rnn, epochs=2, batch_size=32, validation_data=(X_test_rnn, y_test_rnn))
rnn_accuracy = rnn_model.evaluate(X_test_rnn, y_test_rnn, verbose=0)[1]
print("RNN Accuracy:", rnn_accuracy)


Epoch 1/2
430/430 ━━━━━━━━━━━━━━━━━━━━ 34s 75ms/step - accuracy: 0.7500 - loss: 0.5201 - val_accuracy: 0.7904 - val_loss: 0.4683
Epoch 2/2
430/430 ━━━━━━━━━━━━━━━━━━━━ 40s 73ms/step - accuracy: 0.7941 - loss: 0.4073 - val_accuracy: 0.8236 - val_loss: 0.3363
RNN Accuracy: 0.823580801486969


In [ ]:
# Simple GAN Generator
def build_generator(latent_dim):
    model = models.Sequential([
        layers.Dense(128 * 32 * 32, activation="relu", input_dim=latent_dim),
        layers.Reshape((32, 32, 128)),
        layers.UpSampling2D(),
        layers.Conv2D(128, kernel_size=3, padding="same"),
        layers.BatchNormalization(momentum=0.8),
        layers.Activation("relu"),
        layers.UpSampling2D(),
        layers.Conv2D(1, kernel_size=3, padding="same"),
        layers.Activation("tanh")
    ])
    return model

latent_dim = 100
generator = build_generator(latent_dim)

#  GAN sample generation + CNN retraining
import gc
import tensorflow.keras.backend as K

def generate_and_train_batchwise(cnn_model, generator, X_train, y_train, latent_dim=100, batch_size=64, steps=10):
    for step in range(steps):
        idx = np.random.randint(0, len(X_train), batch_size)
        real_imgs = X_train[idx]
        real_labels = y_train[idx]

        noise = np.random.normal(0, 1, (batch_size, latent_dim))
        fake_imgs = generator.predict(noise, verbose=0)
        fake_imgs_resized = np.array([cv2.resize(img, (128, 128)).reshape(128, 128, 1) for img in fake_imgs])

        combined_imgs = np.concatenate((real_imgs, fake_imgs_resized), axis=0)
        combined_labels = np.concatenate((real_labels, np.ones((batch_size,))), axis=0)

        cnn_model.fit(combined_imgs, combined_labels, epochs=1, batch_size=batch_size, verbose=1)

        del fake_imgs, fake_imgs_resized, combined_imgs, combined_labels
        K.clear_session()
        gc.collect()

    final_acc = cnn_model.evaluate(X_test, y_test, verbose=0)[1]
    print("CNN Accuracy after GAN augmentation:", final_acc)
    return final_acc

#  GAN + CNN training
generate_and_train_batchwise(cnn_model, generator, X_train, y_train, latent_dim=latent_dim, batch_size=64, steps=10)


2/2 ━━━━━━━━━━━━━━━━━━━━ 4s 2s/step - accuracy: 0.9740 - loss: 0.0823
2/2 ━━━━━━━━━━━━━━━━━━━━ 5s 3s/step - accuracy: 1.0000 - loss: 0.0326
2/2 ━━━━━━━━━━━━━━━━━━━━ 4s 2s/step - accuracy: 0.9688 - loss: 0.0409
2/2 ━━━━━━━━━━━━━━━━━━━━ 5s 3s/step - accuracy: 0.9635 - loss: 0.0629
2/2 ━━━━━━━━━━━━━━━━━━━━ 3s 2s/step - accuracy: 0.9479 - loss: 0.0926
2/2 ━━━━━━━━━━━━━━━━━━━━ 3s 2s/step - accuracy: 1.0000 - loss: 0.0143
2/2 ━━━━━━━━━━━━━━━━━━━━ 4s 2s/step - accuracy: 0.9740 - loss: 0.0328
2/2 ━━━━━━━━━━━━━━━━━━━━ 4s 2s/step - accuracy: 0.9896 - loss: 0.0342
2/2 ━━━━━━━━━━━━━━━━━━━━ 5s 2s/step - accuracy: 0.9896 - loss: 0.0244
2/2 ━━━━━━━━━━━━━━━━━━━━ 3s 2s/step - accuracy: 0.9792 - loss: 0.0264
CNN Accuracy after GAN augmentation: 0.9930130839347839


0.9930130839347839